## Dashboard Interactivo del Modelo Mundell-Fleming

Este notebook contiene el código para generar un dashboard interactivo del modelo Mundell-Fleming. Permite visualizar los efectos de políticas fiscales, monetarias y shocks cambiarios en una economía abierta con tipo de cambio flexible.

#   ----------------------------------------------------------------------------
#   Programación I 
#   HW3: coloque las funciones utilizadas en los ejercicios. No
#   defina otras variables o valores en este módulo.
#   ----------------------------------------------------------------------------
import numpy as np
import pandas as pd

# ----------------------------------------------
# Ejercicio 1
# ----------------------------------------------
def load_mean(filename: str):
    arch = np.load(filename)
    mean_value = arch.mean()
    return mean_value

# ----------------------------------------------
# Ejercicio 2
# ----------------------------------------------
# Recibe una matriz X con las variables en las columnas y observaciones en las
# filas. El vector columna "y" debe tener la misma cantidad de observaciones  
def ols_estimation(X: np.ndarray, y: np.ndarray):
    """
    Computes OLS coefficient estimates.
    """
    # Add an intercept column of ones to the design matrix X
    intercept = np.ones((X.shape[0], 1))
    design_matrix = np.hstack((intercept, X))
    
    # Compute beta_hat using the OLS formula: (X'X)^-1 * X'y
    beta_hat = np.linalg.inv(design_matrix.T @ design_matrix) @ design_matrix.T @ y
    
    return beta_hat

# ----------------------------------------------
# Ejercicio 3
# ----------------------------------------------
# Devuelve los puntos que anotaría Curry en un cuarto de juego (1 realización)
def curry_one_quarter():
    total_points = 0
    num_intervals = 20  # 10 minutes / 30 seconds per interval

    # Probabilities for each type of play in an interval
    play_types = ['2pt', '3pt', '0pts']
    probabilities = [1/4, 1/3, 1 - (1/4 + 1/3)]

    for _ in range(num_intervals):
        play = np.random.choice(play_types, p=probabilities)
        prob_score = 0
        if play == '2pt':
            # Time T is exponential with mean (scale) = 5 seconds
            T = np.random.exponential(scale=5)
            prob_score = 4/5 if T < 3.2 else 1/5
            if np.random.rand() < prob_score:
                total_points += 2
        elif play == '3pt':
            # 75% chance to score  
            if np.random.rand() < 0.75:
                total_points += 3

    return total_points

# ----------------------------------------------
# Ejercicio 4
# ----------------------------------------------
def logistic(x0, r, n=100):
    x = np.zeros(n)
    x[0] = x0
    for i in range(n - 1):
        x[i+1] = r * x[i] * (1 - x[i])
    return x

def stable_values(r, x0=0.25, n=500):
    trajectory = logistic(x0, r, n)
    # Get last 100 values and round to handle float precision issues
    last_values = trajectory[-100:]
    # Find unique values among the rounded stable states
    unique_stable_values = np.unique(np.round(last_values, decimals=8))
    return unique_stable_values

# ----------------------------------------------
# Ejercicio 5
# ----------------------------------------------
# Number of unique countries

df = pd.read_csv("bigmac.csv", index_col=0) 

def get_number_unique_countries(df: pd.DataFrame) -> int:     
    return df["name"].nunique()

# Top 10 countries with highest local price
def get_top10_local_price(df: pd.DataFrame) -> set:
    july = df.query("date == '2024-07-01'")    
    return set(july.sort_values("local_price", ascending=False).head(10)["name"])

# Median of prices in july 2024
def get_median(df: pd.DataFrame) -> float: 
    july = df.query("date == '2024-07-01'")
    return float(july["adj_price"].median())

# Mean adjusted prices for a given country
def get_mean_adj_price(df: pd.DataFrame, country: str) -> float:
    country_data = df[df['name'] == country]
    return float(country_data['adj_price'].mean())